# Evo-Bayes-Agent

In [25]:
from Ionomy import IonPanda, BitTA
from decouple import config

MARKET = 'usd-btc'
CURRENCY = 'btc'
BASE = 'usd'
TIME = 'hour'
initial_capital = 5_000
MAX_BUY = 0.1
MAX_SELL = 0.1
FEE_RATE = 0.01

window_size = 32
layer_size = 500
output_size = 3

bta = BitTA(config('TREX_KEY'), config('TREX_SECRET'))
ionpd = IonPanda(config('IONOMY_KEY'), config('IONOMY_SECRET'))
bta.update(CURRENCY, BASE, TIME)
ohlcv_df = bta.df

In [26]:
import array
from numpy import random
import numpy as np
import pandas as pd
import pandas_ta as ta

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from deap import benchmarks
from deap import cma

In [27]:
def init_ind():
    signal_weights = [
        np.random.randn(window_size, layer_size),
        np.random.randn(layer_size, output_size),
    ]
    signal_biases = [
        np.random.randn(layer_size),
        np.random.randn(output_size),
    ]
    return np.array([signal_weights, signal_biases])

In [42]:
def act(state, nn):
    weights = nn[0]
    biases = nn[1]
    action = state
    for layer, bias in zip(weights, biases):
        action = np.dot(action, layer) + bias
    return int(np.around(np.argmax(action)))

In [49]:
def fitness(individual, ohlcv_df):
    balance = initial_capital
    position = 0
    for time_index in range(len(ohlcv_df)-1):
        if time_index < window_size:
            continue
        state = ohlcv_df["close"].pct_change().loc[time_index-window_size:time_index-1].to_numpy(copy=True)
        signal = act(state, individual)
        current_price = ohlcv_df.loc[time_index, "close"]
        if signal == 1:
            total_buy = MAX_BUY * current_price
            if total_buy > balance:
                continue
            fee = FEE_RATE*total_buy
            balance -= (total_buy + fee)
            position += MAX_BUY
        elif signal == 2 and position > 0:
            position -= MAX_SELL
            total_sell = MAX_SELL * current_price
            fee = FEE_RATE*total_sell
            balance += (total_sell - fee)
    return ((balance - initial_capital) / initial_capital) * 100,

In [52]:
N=5
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("evaluate", fitness, ohlcv_df=ohlcv_df)

def evolve(input_size, layer_size, output_size):
    np.random.seed()

    parent = creator.Individual(init_ind())
    parent.fitness.values = toolbox.evaluate(parent)
    
    strategy = cma.StrategyOnePlusLambda(parent, sigma=5.0, lambda_=10)
    toolbox.register("generate", strategy.generate, ind_init=creator.Individual)
    toolbox.register("update", strategy.update)

    hof = tools.HallOfFame(1)    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
   
    pop, logbook = algorithms.eaGenerateUpdate(toolbox, ngen=200, halloffame=hof, stats=stats)
    return pop, logbook, hof

In [ ]:
INDIVIDUAL_SIZE = 50

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=INDIVIDUAL_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxUniform)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=5)
def main():
    pop = toolbox.population(n=100)
    mate_probability = 0.5
    mutation_probability = 0.2
    number_generations = 40

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop) 
    # Set fitnesses to individuals
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
 
    # loop for the number of generation we need
    for g in range(number_generations):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = map(toolbox.clone, offspring)

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < mate_probability:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < mutation_probability:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop

In [53]:
evolve(window_size, layer_size, output_size)

ValueError: operands could not be broadcast together with shapes (2,2) (10,2) 